In [1]:
import os
import sys
import cv2

import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.init as init
import torchvision

In [2]:
PATH = os.path.join('..', 'input')

In [3]:
os.listdir(PATH)

['test_images',
 '.DS_Store',
 'test.csv',
 'train.csv',
 'train_images',
 'sample_submission.csv']

In [4]:
train = pd.read_csv(os.path.join(PATH, 'train.csv'))

In [5]:
train.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


In [6]:
test = pd.read_csv(os.path.join(PATH, 'test.csv'))

In [7]:
test.head()

,id_code
0,0005cfc8afb6
1,003f0afdcd15
2,006efc72b638
3,00836aaacf06
4,009245722fa4
